In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch import tensor
from torchvision import transforms, utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import pandas as pd
import pydicom as dicom
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
from PIL import Image


from glob import glob
import os


Bad key "text.kerning_factor" on line 4 in
C:\Users\lasse\.conda\envs\pytorch\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
if torch.cuda.is_available():
    # you can continue going on here, like cuda:1 cuda:2....etc.
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
DATA_DIR = os.path.join(".", "rsa-pneumonia-data")

TRAIN_IMAGES = os.path.join(DATA_DIR, "stage_2_train_images")
TEST_IMAGES = os.path.join(DATA_DIR, "stage_2_test_images")

TRAIN_ANNOTATIONS_FILE = "stage_2_train_labels.csv"
TRAIN_CLASS_FILE = "stage_2_detailed_class_info.csv"

In [4]:
class OpacityDataset(Dataset):

    def __init__(self, csv_file, root_dir, image_dir, subsample, transform=None):
        self.classes_df = pd.read_csv(csv_file)
        if subsample is not None:
            self.classes_df = self.classes_df[:subsample]
        self.root_dir = root_dir
        self.image_dir = image_dir
        self.transform = transform

        # Convert our classes to integers!
        self.class_dict = {
            "Normal": 0,
            "No Lung Opacity / Not Normal": 0,
            "Lung Opacity": 1
        }

    def __len__(self):
        return len(self.classes_df)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        # Get the image, preprocess it for our model
        image_path = os.path.join(
            self.image_dir, self.classes_df.iloc[index, 0])
        image = dicom.read_file(image_path + ".dcm").pixel_array
        image = image[::4, ::4]
        image = image/image.max()
        image = (255*image).clip(0, 255).astype(np.uint8)
        image = Image.fromarray(image).convert("RGB")
        opacity_class = self.classes_df.at[index, 'class']

        if self.transform:
            image = self.transform(image)
        
        sample = {"image": image,
                  "opacity_class": self.class_dict[opacity_class]}

        return sample

In [5]:
validation_split = 0.2;

opacity_dataset = OpacityDataset(
    csv_file=os.path.join(DATA_DIR, TRAIN_CLASS_FILE),
    root_dir=DATA_DIR,
    image_dir=TRAIN_IMAGES,
    subsample=8000,
#     subsample=None,
    transform=transforms.ToTensor()
)

dataset_size = len(opacity_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# np.random.seed(1)
# np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Create samplers for data loading
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [6]:
# Define our NN
# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         self.conv1 = nn.Conv2d(1, 16, (3, 3))
#         self.pool = nn.MaxPool2d((2, 2), 2)
#         self.conv2 = nn.Conv2d(16, 32, (3, 3))
#         self.conv3 = nn.Conv2d(32, 64, (3, 3))
#         self.conv4 = nn.Conv2d(64, 128, (3, 3))
#         self.linear1 = nn.Linear(25088, 4086)
#         self.linear3 = nn.Linear(4086, 2)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = self.pool(F.relu(self.conv4(x)))
#         x = x.view(x.size()[0], -1)
#         x = F.relu(self.linear1(x))
#         x = self.linear3(x)
#         return x
    
# class AlexNet(nn.Module):
#     def __init__(self, num_classes=2):
#         super(AlexNet, self).__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(64, 192, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#             nn.Conv2d(192, 384, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(384, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=3, stride=2),
#         )
#         self.classifier = nn.Sequential(
#             nn.Dropout(),
#             nn.Linear(256 * 7 * 7, 4096),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(4096, 4096),
#             nn.ReLU(inplace=True),
#             nn.Linear(4096, num_classes),
#         )

#     def forward(self, x):
#         x = self.features(x)
#         x = x.view(x.size(0), 256 * 7 * 7)
#         x = self.classifier(x)
#         return x

# model = AlexNet()
# model.to(device)

In [7]:
del model
torch.cuda.ipc_collect()
torch.cuda.empty_cache()

In [7]:
# Hyperparameters
epochs = 10
num_classes = 2
batch_size = 8
learning_rate = .001

In [8]:
model = torchvision.models.vgg11(pretrained=False)
model.fc = nn.Linear(4096, num_classes)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [9]:
# model = torchvision.models.resnet18(pretrained=False)
# feature_num = model.fc.in_features
# model.fc = nn.Linear(feature_num, num_classes)

In [9]:
train_loader = torch.utils.data.DataLoader(
    opacity_dataset, 
    batch_size=batch_size, 
    sampler=train_sampler,
    num_workers=0
)
validation_loader = torch.utils.data.DataLoader(
    opacity_dataset, 
    batch_size=batch_size,
    sampler=valid_sampler,
    num_workers=0
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [10]:
def get_num_from_tensor(tensorItem):
    return tensorItem.cuda().cpu().numpy().item()

In [11]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim=1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim=1)

    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = acc * 100

    return get_num_from_tensor(acc)

In [12]:
losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_train_acc = 0.0
    running_val_acc = 0.0
    running_loss_full = 0.0
    
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data["image"], data["opacity_class"]
        inputs, labels = Variable(inputs.cuda(), requires_grad=True), Variable(labels.cuda())
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches          
            train_acc = multi_acc(outputs, labels)
            print("[%d, %5d] loss: %.3f training_accuracy: %.2f" % (epoch + 1, i + 1, running_loss / 50, train_acc))
            running_loss = 0.0
       
    print("Finished training")
        
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data["image"], data["opacity_class"]
        inputs, labels = Variable(inputs.cuda(), requires_grad=True), Variable(labels.cuda())
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss_full += loss.item()
        running_train_acc += multi_acc(outputs, labels)
        
    for i, data in enumerate(validation_loader, 0):
        inputs, labels = data["image"], data["opacity_class"]
        inputs, labels = Variable(inputs.cuda(), requires_grad=True), Variable(labels.cuda())
        outputs = model(inputs)
        running_val_acc += multi_acc(outputs, labels)
        
    curr_train_loss = running_loss_full / len(train_loader)    
    train_acc = running_train_acc / len(train_loader)
    val_acc = running_val_acc / len(validation_loader)
    
    losses.append(curr_train_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
        
    print("Epoch [%d] Training Accuracy: %.3f Validation Accuracy: %.3f " % (epoch + 1, train_acc, val_acc))            
print('Finished Training')

[1,    50] loss: 3.495 training_accuracy: 37.50
[1,   100] loss: 0.932 training_accuracy: 100.00
[1,   150] loss: 0.785 training_accuracy: 62.50
[1,   200] loss: 0.757 training_accuracy: 75.00
[1,   250] loss: 0.719 training_accuracy: 37.50
[1,   300] loss: 0.694 training_accuracy: 37.50
[1,   350] loss: 0.698 training_accuracy: 50.00
[1,   400] loss: 0.722 training_accuracy: 50.00
[1,   450] loss: 0.693 training_accuracy: 50.00
[1,   500] loss: 0.722 training_accuracy: 37.50
[1,   550] loss: 0.686 training_accuracy: 50.00
[1,   600] loss: 0.692 training_accuracy: 75.00
[1,   650] loss: 0.669 training_accuracy: 62.50
[1,   700] loss: 0.699 training_accuracy: 62.50
[1,   750] loss: 0.687 training_accuracy: 37.50
[1,   800] loss: 0.681 training_accuracy: 75.00
Finished training
Epoch [1] Training Accuracy: 64.656 Validation Accuracy: 56.188 
[2,    50] loss: 0.669 training_accuracy: 62.50
[2,   100] loss: 0.668 training_accuracy: 75.00
[2,   150] loss: 0.685 training_accuracy: 87.50
[2, 

[10,   600] loss: 0.459 training_accuracy: 62.50
[10,   650] loss: 0.492 training_accuracy: 75.00
[10,   700] loss: 0.470 training_accuracy: 62.50
[10,   750] loss: 0.501 training_accuracy: 62.50
[10,   800] loss: 0.493 training_accuracy: 62.50
Finished training
Epoch [10] Training Accuracy: 74.109 Validation Accuracy: 70.562 
Finished Training


In [13]:
df = pd.DataFrame(zip(train_accuracies, val_accuracies), columns=["Train Acc", "Val Acc"])
df.to_csv("vgg16.csv")

In [ ]:
df2 = pd.DataFrame(zip(train_accuracies, val_accuracies), columns=["Train Acc", "Val Acc"])
df2.head()

In [ ]:
torch.save(model.state_dict(), 'resnet18-10-32.pth')

In [ ]:
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.ylabel("Accuracy")
plt.ylim(0,100)
plt.legend()
plt.show()

## Notes
### 10 Epochs, 32 Batch, Resnet18
* Validation values didn't increase that much past the first epoch
* Need to check class imbalance 
* Need to check the amount of values of each class that I'm missing
* Try other models with dropout 
* Clear indiciation of overfitting